# Accessing ERA5 and HadUK-Grid weather data

Info about HadUK-Grid
* Met Office Web page intro https://www.metoffice.gov.uk/research/climate/maps-and-data/data/haduk-grid/overview
* List of components https://www.metoffice.gov.uk/research/climate/maps-and-data/data/haduk-grid/datasets
* Data access through CEDA archive https://catalogue.ceda.ac.uk/uuid/4dc8450d889a491ebb20e724debe2dfb
* Paper on dataset - https://rmets.onlinelibrary.wiley.com/doi/full/10.1002/gdj3.78

Info about ECMWF ERA5 and acessing:
* ECMWF web page https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5
* Access through AWS https://registry.opendata.aws/ecmwf-era5/
* Access through CEDA https://catalogue.ceda.ac.uk/uuid/7b64cf24f47f4b1aa499339c5a576be1 (available soon)
* Notebook demoing access through AWS https://github.com/planet-os/notebooks/blob/master/aws/era5-s3-via-boto.ipynb

In [1]:
# Initialize notebook environment.
%matplotlib inline
import boto3
import botocore
import datetime
import matplotlib.pyplot as plt
import os.path


In [2]:
import xarray as xr

In [3]:
import matplotlib.pyplot

In [5]:
era5_bucket = 'era5-pds'

# AWS access / secret keys required
s3 = boto3.resource('s3')
bucket = s3.Bucket(era5_bucket)

# No AWS keys required
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

In [6]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=era5_bucket, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

1979/
1980/
1981/
1982/
1983/
1984/
1985/
1986/
1987/
1988/
1989/
1990/
1991/
1992/
1993/
1994/
1995/
1996/
1997/
1998/
1999/
2000/
2001/
2002/
2003/
2004/
2005/
2006/
2007/
2008/
2009/
2010/
2011/
2012/
2013/
2014/
2015/
2016/
2017/
2018/
2019/
2020/
2021/
QA/
zarr/


In [7]:
keys = []
date = datetime.date(2019,1,1) # update to desired date
prefix = date.strftime('%Y/%m/')

response = client.list_objects_v2(Bucket=era5_bucket, Prefix=prefix)
response_meta = response.get('ResponseMetadata')

if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None:
        print("No objects are available for %s" % date.strftime('%B, %Y'))
    else:
        for obj in contents:
            keys.append(obj.get('Key'))
        print("There are %s objects available for %s\n--" % (len(keys), date.strftime('%B, %Y')))
        for k in keys:
            print(k)
else:
    print("There was an error with your request.")

There are 19 objects available for January, 2019
--
2019/01/data/air_pressure_at_mean_sea_level.nc
2019/01/data/air_temperature_at_2_metres.nc
2019/01/data/air_temperature_at_2_metres_1hour_Maximum.nc
2019/01/data/air_temperature_at_2_metres_1hour_Minimum.nc
2019/01/data/dew_point_temperature_at_2_metres.nc
2019/01/data/eastward_wind_at_100_metres.nc
2019/01/data/eastward_wind_at_10_metres.nc
2019/01/data/integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation.nc
2019/01/data/lwe_thickness_of_surface_snow_amount.nc
2019/01/data/northward_wind_at_100_metres.nc
2019/01/data/northward_wind_at_10_metres.nc
2019/01/data/precipitation_amount_1hour_Accumulation.nc
2019/01/data/sea_surface_temperature.nc
2019/01/data/sea_surface_wave_from_direction.nc
2019/01/data/sea_surface_wave_mean_period.nc
2019/01/data/significant_height_of_wind_and_swell_waves.nc
2019/01/data/snow_density.nc
2019/01/data/surface_air_pressure.nc
2019/01/main.nc


In [8]:
import s3fs

In [9]:
fs1 = s3fs.S3FileSystem()

In [10]:
path1 = f's3://{era5_bucket}/2019/01/data/air_temperature_at_2_metres.nc'

In [11]:
fileObj1 = fs1.open(path1)

In [12]:
summary_ds = xr.open_dataset(fileObj1, engine='h5netcdf')

In [13]:
temp_hourly_cube = summary_ds.air_temperature_at_2_metres.to_iris()

In [17]:
temp_hourly_cube[:30].interpolate([('latitude', 51.0),('longitude', -3.1)],iris.analysis.Linear()).data

masked_array(data=[282.1624755859375, 282.375, 282.3500061035156,
                   282.5625, 282.26251220703125, 282.1000061035156, 282.0,
                   281.5, 280.9250183105469, 280.4875183105469,
                   281.3375244140625, 282.48748779296875, 283.125,
                   283.4375, 283.2874755859375, 282.75, 282.6499938964844,
                   282.07501220703125, 281.98748779296875,
                   282.32501220703125, 281.8125, 280.9125061035156,
                   279.8999938964844, 278.8375244140625,
                   278.3500061035156, 277.6499938964844,
                   277.67498779296875, 276.9750061035156,
                   276.20001220703125, 275.3500061035156],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False],
       f

In [ ]:
uk_temp_hourly = temp_hourly_cube.intersection(latitude=(49,60), longitude=(-7, 3))

In [ ]:
fig1 = matplotlib.pyplot.figure(figsize=(8,16))
ax1 = fig1.add_subplot(1,1,1,projection=cartopy.crs.PlateCarree())
iris.quickplot.contourf(uk_temp_hourly[0,:,:],axes=ax1)
ax1.coastlines()

In [ ]:
5%2